#  <center>**BAB IV : PERSAMAAN LINIER**</center> 

**Dhiya Salma Salsabila/24923304**

## **Metode Langsung**

Metode langsung adalah metode yang secara teoritis memberikan solusi yang eksak untuk sistem dalam jumlah langkah yang terbatas.

### **Gaussian elimination with backward substitution**


In [1]:
import numpy as np

def gaussian_elimination(A, b):
    A = A.astype(float)  # Mengonversi matriks A menjadi float64
    b = b.astype(float)  # Mengonversi vektor b menjadi float64
    n = len(b)
    # Langkah Eliminasi
    for k in range(n-1):
        # Pengecekan elemen diagonal utama
        if A[k, k] == 0:
            # Cari baris lain dengan elemen non-nol di kolom k untuk ditukar
            for i in range(k+1, n):
                if A[i, k] != 0:
                    A[[k, i]] = A[[i, k]]  # Tukar baris
                    b[[k, i]] = b[[i, k]]
                    break
        # Hitung faktor pengali dan lakukan eliminasi
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]
            A[i, k+1:] -= m * A[k, k+1:]
            b[i] -= m * b[k]
            
    # Substitusi Mundur
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

# Contoh
A = np.array([[1, 1, 0, 3],
              [2, 1, -1, 1],
              [3, -1, -1, 2],
              [-1, 2, 3, -1]])
b = np.array([4, 1, -3, 4])

x = gaussian_elimination(A, b)
print("Solusi x:", x)

Solusi x: [-1.  2.  0.  1.]


### **Pivoting Parsial**

In [2]:
import numpy as np

def partial_pivot(A, b):
    n = len(A)
    for k in range(n-1):
        max_index = np.argmax(abs(A[k:, k])) + k  # Cari indeks baris dengan elemen terbesar di kolom k
        if max_index != k:
            A[[k, max_index]] = A[[max_index, k]]  # Tukar baris k dengan baris max_index
            b[[k, max_index]] = b[[max_index, k]]
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]  # Kurangi baris i dengan m kali baris k
            b[i] -= m * b[k]

def back_substitution(A, b):
    n = len(A)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):  # Mulai dari baris terakhir dan mundur
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

def gaussian_elimination(A, b):
    partial_pivot(A, b)
    return back_substitution(A, b)

# Contoh penggunaan
A = np.array([[0.003, 59.14], [5.291, -6.130]], dtype=float)
b = np.array([59.17, 46.78], dtype=float)

x = gaussian_elimination(A, b)
print("Solusi x:", x)

Solusi x: [10.  1.]


### **Scaled Python Partial Pivoting**

In [3]:
import numpy as np

def scaled_partial_pivoting(A, b):
    n = len(b)
    s = np.zeros(n)
    for i in range(n):
        s[i] = max(abs(A[i, :]))
    for k in range(n-1):
        # Pilih baris pivot berdasarkan skala relatif
        pivot_row = np.argmax(abs(A[k:, k]) / s[k:]) + k
        
        # Tukar baris jika perlu
        if pivot_row != k:
            A[[k, pivot_row]] = A[[pivot_row, k]]
            b[[k, pivot_row]] = b[[pivot_row, k]]
            s[[k, pivot_row]] = s[[pivot_row, k]]
        
        for i in range(k+1, n):
            # Lakukan eliminasi Gauss
            factor = A[i, k] / A[k, k]
            A[i, k+1:] -= factor * A[k, k+1:]
            b[i] -= factor * b[k]
    # Lakukan substitusi balik untuk mendapatkan solusi
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

# Contoh penggunaan
A = np.array([[30, 591400], [5.291, -6.130]], dtype=float)
b = np.array([591700, 46.78], dtype=float)

x = scaled_partial_pivoting(A, b)
print("Solusi x:", x)

Solusi x: [10.  1.]


### **Faktorisasi LU**

In [4]:
import numpy as np

def LU_decomposition(A):
    n = len(A)
    L = np.eye(n)
    U = np.copy(A)

    for k in range(n-1):
        for i in range(k+1, n):
            if U[k, k] == 0:
                raise ValueError("Pivot nol ditemukan. Faktorisasi LU tidak dapat dilakukan.")
            L[i, k] = U[i, k] / U[k, k]
            for j in range(k, n):
                U[i, j] -= L[i, k] * U[k, j]

    return L, U

# Contoh penggunaan
A = np.array([[1, 1, 0, 3],
              [2, 1, -1, 1],
              [3, -1, -1, 2],
              [-1, 2, 3, -1]])

L, U = LU_decomposition(A)
print("Matriks L:")
print(L)
print("Matriks U:")
print(U)

Matriks L:
[[ 1.  0.  0.  0.]
 [ 2.  1.  0.  0.]
 [ 3.  4.  1.  0.]
 [-1. -3.  0.  1.]]
Matriks U:
[[  1   1   0   3]
 [  0  -1  -1  -5]
 [  0   0   3  13]
 [  0   0   0 -13]]


### **Faktorisasi LDL**

In [5]:
import numpy as np

def LDLT_decomposition(A):
    n = len(A)
    L = np.zeros((n, n))
    d = np.zeros(n)

    for i in range(n):
        d[i] = A[i][i] - np.sum(np.square(L[i][:i]) * d[:i])

        for j in range(i+1, n):
            L[j][i] = (A[j][i] - np.sum(L[j][:i] * L[i][:i] * d[:i])) / d[i]

        L[i][i] = 1

    return L, d

# Example usage:
A = np.array([[2, 4, 6],
              [4, 9, 14],
              [6, 14, 19]])

L, d = LDLT_decomposition(A)
print("Matriks segitiga bawah L:")
print(L)
print("Vektor diagonal d:")
print(d)

Matriks segitiga bawah L:
[[1. 0. 0.]
 [2. 1. 0.]
 [3. 2. 1.]]
Vektor diagonal d:
[ 2.  1. -3.]


### **Faktorisasi Cholesky**

In [6]:
import numpy as np

def cholesky_decomposition(A):
    n = len(A)
    G = np.zeros_like(A, dtype=float)
    
    for i in range(n):
        for j in range(i + 1):
            if i == j:
                G[i][j] = np.sqrt(A[i][i] - np.sum(G[i][k]**2 for k in range(j)))
            else:
                G[i][j] = (A[i][j] - np.sum(G[i][k] * G[j][k] for k in range(j))) / G[j][j]
        
        if G[j][j] <= 0:
            print("Matrix is not positive definite.")
            return None
    
    return G

# Example usage:
A = np.array([[4, 4, 6],
              [4, 5, 8],
              [6, 8, 22]])

G = cholesky_decomposition(A)
if G is not None:
    print("Lower triangular matrix G:")
    print(G)

Lower triangular matrix G:
[[2. 0. 0.]
 [2. 1. 0.]
 [3. 2. 3.]]


C:\Users\Asus\AppData\Local\Temp\ipykernel_12736\3755568674.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  G[i][j] = np.sqrt(A[i][i] - np.sum(G[i][k]**2 for k in range(j)))
C:\Users\Asus\AppData\Local\Temp\ipykernel_12736\3755568674.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  G[i][j] = (A[i][j] - np.sum(G[i][k] * G[j][k] for k in range(j))) / G[j][j]


### **Faktorisasi LU untuk matriks tridiagonal**

In [7]:
import numpy as np

def solve_LU_system(n, A):
    # Step 1
    L = np.zeros((n, n))
    U = np.eye(n)
    l11 = A[0, 0]
    u12 = A[0, 1] / l11
    z1 = A[0, -1] / l11
    
    # Step 2
    L[0, 0] = l11
    U[0, 1] = u12
    z = np.zeros(n)
    z[0] = z1
    for i in range(1, n-1):
        L[i, i-1] = A[i, i-1]
        L[i, i] = A[i, i] - L[i, i-1] * U[i-1, i]
        U[i, i+1] = A[i, i+1] / L[i, i]
        z[i] = (A[i, -1] - L[i, i-1] * z[i-1]) / L[i, i]
    
    # Step 3
    L[n-1, n-2] = A[n-1, n-2]
    L[n-1, n-1] = A[n-1, n-1] - L[n-1, n-2] * U[n-2, n-1]
    z[n-1] = (A[n-1, -1] - L[n-1, n-2] * z[n-2]) / L[n-1, n-1]
    
    # Step 4
    x = np.zeros(n)
    x[n-1] = z[n-1]
    
    # Step 5
    for i in range(n-2, -1, -1):
        x[i] = z[i] - U[i, i+1] * x[i+1]
    
    return x, L, U

# Contoh penggunaan
n = 4
A = np.array([[2, -1, 0, 0], [-1, 2, -1, 0], [0, -1, 2, -1], [0, 0, -1, 2]])
x, L, U = solve_LU_system(n, A)
print("Solusi x:", x)
print("Matriks L:")
print(L)
print("Matriks U:")
print(U)

Solusi x: [0. 0. 0. 1.]
Matriks L:
[[ 2.          0.          0.          0.        ]
 [-1.          1.5         0.          0.        ]
 [ 0.         -1.          1.33333333  0.        ]
 [ 0.          0.         -1.          1.25      ]]
Matriks U:
[[ 1.         -0.5         0.          0.        ]
 [ 0.          1.         -0.66666667  0.        ]
 [ 0.          0.          1.         -0.75      ]
 [ 0.          0.          0.          1.        ]]


## **Metode Iteratif**

Teknik iterasi untuk menyelesaikan sistem linier $A\vec{x} = \vec{b}$ dimulai dengan aproksimasi awal $\vec{x}^{(0)}$ ke solusi $\vec{x}$ dan menghasilkan barisan vektor  $ \{ \vec{x}^{(0)},  \vec{x}^{(1)},  \vec{x}^{(2)}, ... \}$  yang konvergen ke $\vec{x}$. Metode iterasi mempunyai kelebihan, yaitu efisiensi memori dan waktu komputasi CPU.

### **Metode Jacobi**

In [8]:
import numpy as np

def jacobi_iteration(A, b, X0, TOL, N):
    n = len(b)
    X = X0.copy()
    k=1
    for k in range(N):
        X_new = np.zeros(n)
        for i in range(n):
            sigma = sum([A[i, j] * X[j] for j in range(n) if j != i])
            X_new[i] = round((b[i] - sigma) / A[i, i],4)
        if np.linalg.norm(X_new - X, ord=2)/np.linalg.norm(X_new, ord=2) < TOL:
            return X_new
        X = X_new
    print("Maximum number of iterations exceeded")
    return X

# Example:
A = np.array([[10, -1, 2, 0], [-1, 11, -1, 3], [2, -1, 10, -1], [0, 3, -1, 8]])
b = np.array([6, 25, -11, 15])
X0 = np.zeros_like(b)
TOL = 1e-3
N = 1000

solution = jacobi_iteration(A, b, X0, TOL, N)
print("Approximate solution:", solution)

Approximate solution: [ 1.0001  1.9998 -0.9998  0.9998]


### **Metode Gauss-Seidel**

In [9]:
import numpy as np

def solve_linear_system(n, A, b, XO, TOL, N):
    k = 1
    while k <= N:
        x = np.zeros(n)
        for i in range(n):
            x[i] = round((b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i+1:], XO[i+1:])) / A[i, i],4)
        
        if np.linalg.norm(x - XO,ord=2)/np.linalg.norm(x, ord=2) < TOL:
            return x  # Jika solusi sudah mendekati XO dengan toleransi TOL, kembalikan solusi
        k += 1
        XO = x
    
    return "Jumlah iterasi maksimum telah terlampaui"

# Contoh penggunaan
n = len(b)
A = np.array([[10, -1, 2, 0], [-1, 11, -1, 3], [2, -1, 10, -1], [0, 3, -1, 8]])
b = np.array([6, 25, -11, 15])
XO = np.zeros(n)
TOL = 1e-3
N = 1000

solution = solve_linear_system(n, A, b, XO, TOL, N)
print("Solusi x:", solution)

Solusi x: [ 1.0001  2.     -1.      1.    ]


### **Metode Successive Over-Relaxation (SOR)**

In [10]:
import numpy as np

def SOR(A, b, XO, ω, TOL, N):
    n = len(A)
    x = np.copy(XO)
    k = 1
    
    while k <= N:
        for i in range(n):
            sum1 = np.dot(A[i, :i], x[:i])
            sum2 = np.dot(A[i, i+1:], XO[i+1:])
            x[i] = (1 - ω) * XO[i] + (ω / A[i, i]) * (b[i] - sum1 - sum2)
            
        if np.linalg.norm(x - XO) < TOL:
            return x  # The procedure was successful
    
        k += 1
        XO = np.copy(x)
    
    print('Maximum number of iterations exceeded')
    return None  # The procedure was unsuccessful

# Example usage:
A = np.array([[4, 3, 0],
              [3, 4, -1],
              [0, -1, 4]])
b = np.array([24, 30, -24])
XO = np.zeros_like(b)
ω = 1.25
TOL = 1e-7
N = 7

solution = SOR(A, b, XO, ω, TOL, N)
if solution is not None:
    print("Approximate solution:", solution)

Approximate solution: [ 3  4 -5]
